# Chest X-ray Abnormalities Detection (PREDICTION)

**ECE-GY 9123 | Deep Learning**

**Spring 2021**

**TEAM MEMBERS**

**Kunwar Srivastav** (kss519)

**Sagar Patel** (sp5894)

---

**NOTE:**

This Notebook is the second part of the project where we discuss the prediction using `detectron2`.

Most of the prediction techniques have been inspired from implementations by other profiles on Kaggle.

## Table of Contents

- [Dataset preparation](#Dataset-preparation)
- [Re-installing detectron2](#Re-installing-detectron2)
- [Implementing Prediction methods](#Implementing-Prediction-methods)
    - [Data preparation](#Data-preparation)
    - [Loading the data](#Loading-the-data)
- [Predictions file and outputs](#Predictions-file-and-outputs)
    - [Creating submission.csv](#Creating-submission.csv)
- [Potential extensions](#Potential-extensions)

## Dataset preparation

In [1]:
import gc
import os
from pathlib import Path
import random
import sys

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy as sp


import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

pd.set_option('max_columns', 50)

## Re-installing detectron2

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


In [4]:
import torch

torch.__version__

'1.7.0'

In [5]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.7/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.7/index.html
     |████████████████████████████████| 6.8 MB 73.1 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.3.post20210204-py3-none-any.whl size=44946 sha256=cebd71157a8c63796dd572aa0d25b0c30d288bfad63f3588790ab331d761d912
  Stored in directory: /root/.cache/pip/wheels/0a/f2/8c/124367ec901d4b48b5ba4c0226c0a8239815b4e969ad15cc7a
  Created wheel for iopath: filename=iopath-0.1.3-py3-none-any.whl size=11169 sha256=4cc6176d7d4d20d3d74fa93127008acf6c3e0a6734d7fd12c2b4fb62d2c674ee
  Stored in directory: /root/.cache/pip/wheels/33/44/20/06445612ad8cf4ad6250aee85516e9499d5a49151ef5358164
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=273765 sha256=8c7b0e703bfde63e0c0cb0867ddeb6a26fa0d28d079ca6d26513f8602193ad7b
  Stored in directory: /root/.cache/pip/wheels/bc/cf/1b/e95c99c5f9d1648be3f500ca55e7ce55f24818b0f48336adaf
Successfully built fvcore iopath pycocotool

## Implementing Prediction methods

### Data preparation

In [6]:
import pickle
from pathlib import Path
from typing import Optional

import cv2
import numpy as np
import pandas as pd
from detectron2.structures import BoxMode
from tqdm import tqdm


def get_vinbigdata_dicts(
    imgdir: Path,
    train_df: pd.DataFrame,
    train_data_type: str = "original",
    use_cache: bool = True,
    debug: bool = True,
    target_indices: Optional[np.ndarray] = None,
    use_class14: bool = False,
):
    debug_str = f"_debug{int(debug)}"
    train_data_type_str = f"_{train_data_type}"
    class14_str = f"_14class{int(use_class14)}"
    cache_path = Path(".") / f"dataset_dicts_cache{train_data_type_str}{class14_str}{debug_str}.pkl"
    if not use_cache or not cache_path.exists():
        print("Creating data...")
        train_meta = pd.read_csv(imgdir / "train_meta.csv")
        if debug:
            train_meta = train_meta.iloc[:500]

        # Load one image to get the size of the image
        image_id = train_meta.loc[0, "image_id"]
        image_path = str(imgdir / "train" / f"{image_id}.png")
        image = cv2.imread(image_path)
        resized_height, resized_width, ch = image.shape
        print(f"image shape: {image.shape}")

        dataset_dicts = []
        for index, train_meta_row in tqdm(train_meta.iterrows(), total=len(train_meta)):
            record = {}

            image_id, height, width = train_meta_row.values
            filename = str(imgdir / "train" / f"{image_id}.png")
            record["file_name"] = filename
            record["image_id"] = image_id
            record["height"] = resized_height
            record["width"] = resized_width
            objs = []
            for index2, row in train_df.query("image_id == @image_id").iterrows():
                class_id = row["class_id"]
                if class_id == 14:
                    if use_class14:
                        bbox_resized = [0, 0, resized_width, resized_height]
                        obj = {
                            "bbox": bbox_resized,
                            "bbox_mode": BoxMode.XYXY_ABS,
                            "category_id": class_id,
                        }
                        objs.append(obj)
                    else:
                        pass
                else:
                    h_ratio = resized_height / height
                    w_ratio = resized_width / width
                    bbox_resized = [
                        float(row["x_min"]) * w_ratio,
                        float(row["y_min"]) * h_ratio,
                        float(row["x_max"]) * w_ratio,
                        float(row["y_max"]) * h_ratio,
                    ]
                    obj = {
                        "bbox": bbox_resized,
                        "bbox_mode": BoxMode.XYXY_ABS,
                        "category_id": class_id,
                    }
                    objs.append(obj)
            record["annotations"] = objs
            dataset_dicts.append(record)
        with open(cache_path, mode="wb") as f:
            pickle.dump(dataset_dicts, f)

    print(f"Load from cache {cache_path}")
    with open(cache_path, mode="rb") as f:
        dataset_dicts = pickle.load(f)
    if target_indices is not None:
        dataset_dicts = [dataset_dicts[i] for i in target_indices]
    return dataset_dicts


def get_vinbigdata_dicts_test(
    imgdir: Path, test_meta: pd.DataFrame, use_cache: bool = True, debug: bool = True,
):
    debug_str = f"_debug{int(debug)}"
    cache_path = Path(".") / f"dataset_dicts_cache_test{debug_str}.pkl"
    if not use_cache or not cache_path.exists():
        print("Creating data...")
        if debug:
            test_meta = test_meta.iloc[:500]

        # Load one image to get the size of the image
        image_id = test_meta.loc[0, "image_id"]
        image_path = str(imgdir / "test" / f"{image_id}.png")
        image = cv2.imread(image_path)
        resized_height, resized_width, ch = image.shape
        print(f"image shape: {image.shape}")

        dataset_dicts = []
        for index, test_meta_row in tqdm(test_meta.iterrows(), total=len(test_meta)):
            record = {}

            image_id, height, width = test_meta_row.values
            filename = str(imgdir / "test" / f"{image_id}.png")
            record["file_name"] = filename
            record["image_id"] = image_id
            record["height"] = resized_height
            record["width"] = resized_width
            dataset_dicts.append(record)
        with open(cache_path, mode="wb") as f:
            pickle.dump(dataset_dicts, f)

    print(f"Load from cache {cache_path}")
    with open(cache_path, mode="rb") as f:
        dataset_dicts = pickle.load(f)
    return dataset_dicts

**REFERENCES:**

- [The function predict_batch](https://github.com/sphinx-doc/sphinx/issues/4258)

In [7]:
from math import ceil
from typing import Any, Dict, List

import cv2
import detectron2
import numpy as np
from numpy import ndarray
import pandas as pd
import torch
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode
from detectron2.utils.logger import setup_logger
from detectron2.utils.visualizer import ColorMode, Visualizer
from tqdm import tqdm


def format_pred(labels: ndarray, boxes: ndarray, scores: ndarray) -> str:
    pred_strings = []
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.int64)
        pred_strings.append(f"{label} {score} {xmin} {ymin} {xmax} {ymax}")
    return " ".join(pred_strings)


def predict_batch(predictor: DefaultPredictor, im_list: List[ndarray]) -> List:
    with torch.no_grad():
        inputs_list = []
        for original_image in im_list:
            # Apply pre-processing to the image
            if predictor.input_format == "RGB":
                original_image = original_image[:, :, ::-1]
            height, width = original_image.shape[:2]
            image = original_image
            image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
            inputs = {"image": image, "height": height, "width": width}
            inputs_list.append(inputs)
        predictions = predictor.model(inputs_list)
        return predictions

In [8]:
# Necessary Utility libraries

from pathlib import Path
from typing import Any, Union

import yaml


def save_yaml(filepath: Union[str, Path], content: Any, width: int = 120):
    with open(filepath, "w") as f:
        yaml.dump(content, f, width=width)


def load_yaml(filepath: Union[str, Path]) -> Any:
    with open(filepath, "r") as f:
        content = yaml.full_load(f)
    return content

In [9]:
# Confguring all the classes

thing_classes = [
    "Aortic enlargement",
    "Atelectasis",
    "Calcification",
    "Cardiomegaly",
    "Consolidation",
    "ILD",
    "Infiltration",
    "Lung Opacity",
    "Nodule/Mass",
    "Other lesion",
    "Pleural effusion",
    "Pleural thickening",
    "Pneumothorax",
    "Pulmonary fibrosis"
]
category_name_to_id = {class_name: index for index, class_name in enumerate(thing_classes)}

### Loading the data

In [10]:
# Flags
from dataclasses import dataclass, field
from typing import Dict


@dataclass
class Flags:
    debug: bool = True
    outdir: str = "results/det"

    # Data configurations
    imgdir_name: str = "vinbigdata-chest-xray-resized-png-256x256"
    split_mode: str = "all_train"
    seed: int = 111
    train_data_type: str = "original"
    use_class14: bool = False

    # Training configurations
    iter: int = 10000
    ims_per_batch: int = 2
    num_workers: int = 4
    lr_scheduler_name: str = "WarmupMultiStepLR"
    base_lr: float = 0.00025
    roi_batch_size_per_image: int = 512
    eval_period: int = 10000
    aug_kwargs: Dict = field(default_factory=lambda: {})

    def update(self, param_dict: Dict) -> "Flags":
        for key, value in param_dict.items():
            if not hasattr(self, key):
                raise ValueError(f"[ERROR] Unexpected key for flag = {key}")
            setattr(self, key, value)
        return self

## Predictions file and outputs

All the methods that needed to be intializaed have been initialized

Referred from Multiple Kaggle Profiles

In [11]:
inputdir = Path("/kaggle/input")
traineddir = inputdir / "vinbigdata-alb-aug-512-cos"

# Flags
flags: Flags = Flags().update(load_yaml(str(traineddir/"flags.yaml")))
print("flags", flags)
debug = flags.debug
outdir = Path(flags.outdir)
os.makedirs(str(outdir), exist_ok=True)

# Reading the data
datadir = inputdir / "vinbigdata-chest-xray-abnormalities-detection"
if flags.imgdir_name == "vinbigdata-chest-xray-resized-png-512x512":
    imgdir = inputdir/ "vinbigdata"
else:
    imgdir = inputdir / flags.imgdir_name

# Reading in the data CSV files
test_meta = pd.read_csv(inputdir / "vinbigdata-testmeta" / "test_meta.csv")
sample_submission = pd.read_csv(datadir / "sample_submission.csv")

flags Flags(debug=False, outdir='results/20210125_all_alb_aug_512_cos', imgdir_name='vinbigdata-chest-xray-resized-png-512x512', split_mode='all_train', seed=111, train_data_type='original', use_class14=False, iter=30000, ims_per_batch=2, num_workers=4, lr_scheduler_name='WarmupCosineLR', base_lr=0.001, roi_batch_size_per_image=512, eval_period=2000, aug_kwargs={'HorizontalFlip': {'p': 0.5}, 'RandomBrightnessContrast': {'p': 0.5}, 'ShiftScaleRotate': {'p': 0.5, 'rotate_limit': 10, 'scale_limit': 0.15}})


In [12]:
cfg = get_cfg()
original_output_dir = cfg.OUTPUT_DIR
cfg.OUTPUT_DIR = str(outdir)
print(f"cfg.OUTPUT_DIR {original_output_dir} -> {cfg.OUTPUT_DIR}")

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("vinbigdata_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2

# Let the training initialize from the MODEL ZOO
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = flags.base_lr
cfg.SOLVER.MAX_ITER = flags.iter
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = flags.roi_batch_size_per_image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(thing_classes)


# Evaluation
cfg.MODEL.WEIGHTS = str(traineddir/"model_final.pth")
print("Original thresh", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.0
print("Changed  thresh", cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST)
predictor = DefaultPredictor(cfg)

DatasetCatalog.register(
    "vinbigdata_test", lambda: get_vinbigdata_dicts_test(imgdir, test_meta, debug=debug)
)
MetadataCatalog.get("vinbigdata_test").set(thing_classes=thing_classes)
metadata = MetadataCatalog.get("vinbigdata_test")
dataset_dicts = get_vinbigdata_dicts_test(imgdir, test_meta, debug=debug)

if debug:
    dataset_dicts = dataset_dicts[:100]

results_list = []
index = 0
batch_size = 4

for i in tqdm(range(ceil(len(dataset_dicts) / batch_size))):
    inds = list(range(batch_size * i, min(batch_size * (i + 1), len(dataset_dicts))))
    dataset_dicts_batch = [dataset_dicts[i] for i in inds]
    im_list = [cv2.imread(d["file_name"]) for d in dataset_dicts_batch]
    outputs_list = predict_batch(predictor, im_list)

    for im, outputs, d in zip(im_list, outputs_list, dataset_dicts_batch):
        resized_height, resized_width, ch = im.shape
        if index < 5:
            v = Visualizer(
                im[:, :, ::-1],
                metadata=metadata,
                scale=0.5,
                instance_mode=ColorMode.IMAGE_BW
            )
            out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
            cv2.imwrite(str(outdir / f"pred_{index}.jpg"), out.get_image()[:, :, ::-1])

        image_id, dim0, dim1 = test_meta.iloc[index].values

        instances = outputs["instances"]
        if len(instances) == 0:
            # No finding, let us initialize [14 1 0 0 1 1]
            result = {"image_id": image_id, "PredictionString": "14 1.0 0 0 1 1"}
        else:
            fields: Dict[str, Any] = instances.get_fields()
            pred_classes = fields["pred_classes"]
            pred_scores = fields["scores"]
            pred_boxes = fields["pred_boxes"].tensor

            h_ratio = dim0 / resized_height
            w_ratio = dim1 / resized_width
            pred_boxes[:, [0, 2]] *= w_ratio
            pred_boxes[:, [1, 3]] *= h_ratio

            pred_classes_array = pred_classes.cpu().numpy()
            pred_boxes_array = pred_boxes.cpu().numpy()
            pred_scores_array = pred_scores.cpu().numpy()

            result = {
                "image_id": image_id,
                "PredictionString": format_pred(
                    pred_classes_array, pred_boxes_array, pred_scores_array
                ),
            }
        results_list.append(result)
        index += 1

cfg.OUTPUT_DIR ./output -> results/20210125_all_alb_aug_512_cos
Original thresh 0.05
Changed  thresh 0.0


 37%|███▋      | 1105/3000 [00:00<00:00, 11045.53it/s]

Creating data...
image shape: (512, 512, 3)


  0%|          | 0/750 [00:00<?, ?it/s]

Load from cache dataset_dicts_cache_test_debug0.pkl


/opt/conda/lib/python3.7/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:124: UserWarning:

This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/torch/csrc/utils/python_arg_parser.cpp:882.)

100%|██████████| 750/750 [02:37<00:00,  4.77it/s]


### Creating submission.csv

As per the Kaggle rules, it was asked to show the outputs in the CSV format. So we decided to follow the same.

In [13]:
submission_det = pd.DataFrame(results_list, columns=['image_id', 'PredictionString'])
submission_det.to_csv(outdir/"submission.csv", index=False)
submission_det

,image_id,PredictionString
0,8dec5497ecc246766acfba5a4be4e619,0 0.775484561920166 1010 603 1248 893 13 0.501...
1,287422bed1d9d153387361889619abed,3 0.9862767457962036 666 1289 1865 1820 0 0.78...
2,1d12b94b7acbeadef7d7700b50aa90d4,0 0.8117097616195679 1173 896 1433 1138 3 0.78...
3,6b872791e23742f6c33a08fc24f77365,11 0.3285936415195465 1799 2196 1900 2336 10 0...
4,d0d2addff91ad7beb1d92126ff74d621,0 0.850891649723053 1422 828 1707 1140 3 0.741...
...,...,...
2995,78b44b96b121d6075d7ae27135278e03,0 0.5035402178764343 1036 771 1214 949 11 0.10...
2996,afee8ff90f29b8827d0eb78774d25324,0 0.27024954557418823 1028 714 1243 947 11 0.0...
2997,6e07fab2014be723250f7897ab6e3df2,0 0.990529477596283 1667 801 1972 1131 3 0.977...
2998,690bb572300ef08bbbb7ebf4196099cf,0 0.5590820908546448 1085 689 1337 956 8 0.464...


## Potential extensions

- It is no surprise that there is a lot of room for improvement in this project. In some cases, maybe running a 2-class classifier would make it more accurate and we might not have such a low confidence score.
- It would probably be better to try **including "No finding" class during detection training** (by adding virtual "No finding" boxes, or by adding global classifier together with the detection).
- Making it open to alien data and creating an application out of this.